In [2]:
! pip install rasterio -q

In [3]:
import ee
from opticalflood import opticflood_sk_local, opticflood_s2

In [ ]:
def initialize_earth_engine(gee_project_name):
    try:
        ee.Initialize()
    except Exception:
        ee.Authenticate()
        ee.Initialize(project=gee_project_name, opt_url='https://earthengine-highvolume.googleapis.com')
initialize_earth_engine('your project name')

In [ ]:
# --- AOI and Parameters ---
aoi = ee.Geometry.Polygon([[
[000, 000],
[000, 000],
[000, 000],
[000, 000],
[000, 000],
]])
flood_event_date = "0000-00-00"
lag = 20
cloud_threshold=50,
min_area = 1000

In [ ]:
flood_maps_sk = opticflood_sk_local(
    input_dir="./input",     # folder with your SkySat TIFFs
    out_dir="./output",      # where flood masks will be written
    min_area=2000,           # same as you set for S2
    overwrite=True           # overwrite existing outputs if rerun
)


In [12]:
import geemap
import ee

Map = geemap.Map()

# --- Visualization palette (0 = dry, 1 = wet/flood) ---
flood_vis = {
    'min': 0,
    'max': 1,
    'palette': ['#ffffff', '#0000ff']  # white (dry) to blue (flooded)

}

# --- Convert ImageCollection to list ---
flood_list = flood_maps_s2.toList(flood_maps_s2.size())
count = flood_maps_s2.size().getInfo()

# --- Add each flood mask to the map ---
for i in range(count):
    img = ee.Image(flood_list.get(i))
    date = ee.Date(img.get('system:time_start')).format('YYYY-MM-dd').getInfo()

    # Clip to AOI for display
    mask_clipped = img.select('depth').clip(aoi)  # <--- Only this line is changed

    # Add flood classification
    Map.addLayer(mask_clipped, flood_vis, f'Flood {date}')

# Center and display
Map.centerObject(aoi, 10)
Map


Map(center=[44.545005685298584, 11.83540887399875], controls=(WidgetControl(options=['position', 'transparent_…